In [0]:
%sh
pip install tweepy demoji spacy langdetect
python -m spacy download en_core_web_lg

In [0]:

import tweepy

# Fill in your user-context credentials (OAuth 1.0a or OAuth 2.0 user-context)
client = tweepy.Client(
    consumer_key="IQpcyROD9aIjYQIutoHFlRbxJ",
    consumer_secret="jsHTQ9QqMv57V6HLIm5oKDKab3F3tINTMiIcmp3229LwSvFHm7",
    access_token="2009380165177233408-QMvCZPYIjL16nSHTy9OY36SoAPw5Tu",
    access_token_secret="0Wnc0zUj5qjgWvl7jMH8Ki41DVYNHsRihQNXE9nqVMcqz",
)

try:
    me = client.get_me(user_fields=["name","username"])
    if me and me.data:
        print("✅ Authenticated user:")
        print(f"ID: {me.data.id}")
        print(f"Name: {me.data.name}")
        print(f"Username: {me.data.username}")
    else:
        print("Token valid, but no user data returned.")
except tweepy.errors.Unauthorized as e:
    print("❌ Unauthorized (check tokens/permissions):", e)
except Exception as e:
    print("❌ Unexpected error:", e)


In [0]:

# -*- coding: utf-8 -*-
import time
import tweepy
from tweepy.errors import TooManyRequests, TweepyException

BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAABJl6wEAAAAAsCHD70FFDxYGFjlfgIpSeWVYcmU%3D8siEUTU3NAE3jOly5sqv4tQoHuXDA7ERkp60GJUKoIjmcY7FtE"
# BEARER_TOKEN = dbutils.secrets.get(scope="my-secrets123", key="bearer-token")
client = tweepy.Client(bearer_token=BEARER_TOKEN)

QUERY = '"ICC" lang:en -is:retweet'

def sleep_until_reset_or_backoff(e: TooManyRequests, backoff_sec: int) -> int:
    """
    If the response has x-rate-limit-reset, sleep exactly until then.
    Otherwise, sleep using the current backoff and return the next backoff (capped).
    """
    resp = getattr(e, "response", None)
    if resp and hasattr(resp, "headers"):
        headers = resp.headers
        reset_epoch = headers.get("x-rate-limit-reset")
        remaining = headers.get("x-rate-limit-remaining")
        if reset_epoch:
            now = int(time.time())
            sleep_for = max(int(reset_epoch) - now, 1)
            print(f"429: remaining={remaining}. Sleeping {sleep_for}s until reset...")
            time.sleep(sleep_for + 1)  # small buffer
            return backoff_sec  # unchanged
        else:
            print("429 received but no reset header present. Using backoff...")
    else:
        print("429 without response headers. Using backoff...")

    # Exponential backoff with a cap (e.g., ~15 minutes)
    print(f"Sleeping {backoff_sec}s...")
    time.sleep(backoff_sec)
    next_backoff = min(backoff_sec * 2, 15 * 60)  # cap at 15 minutes
    return next_backoff

def print_first_tweet(query: str) -> bool:
    """
    Attempt to fetch one page and print the first tweet.
    Returns True on success, False otherwise.
    """
    max_retries = 6  # ~ up to 15 min backoff if no headers
    retries = 0
    backoff_sec = 60  # start at 60s when there's no reset header

    while retries <= max_retries:
        try:
            # Single, small page; do not paginate
            resp = client.search_recent_tweets(
                query=query,
                max_results=10,
                tweet_fields=["created_at", "lang", "public_metrics"]
            )
            tweets = resp.data or []
            if not tweets:
                print("No tweets retrieved by the query.")
                return False

            print("At least one tweet retrieved:", tweets[0].text)
            return True

        except TooManyRequests as e:
            retries += 1
            backoff_sec = sleep_until_reset_or_backoff(e, backoff_sec)

        except TweepyException as e:
            # Transient error (network, server, etc.)
            print(f"Tweepy error: {e}. Retrying in 5s...")
            time.sleep(5)
            retries += 1

        except Exception as e:
            print(f"Unexpected error: {e}")
            return False

    print("Max retries reached; still rate-limited or failing. "
          "Likely monthly cap exhausted or shared project quota at zero.")
    return False

if __name__ == "__main__":
    print_first_tweet(QUERY)


In [0]:
import pandas as pd
import time
import tweepy

tweetList = []
max_retries = 5
backoff_factor = 2

retries = 0
while retries < max_retries:
    try:
        for tweet in tweets:
            tweetList.append(tweet.text)
        break
    except tweepy.TooManyRequests as e:
        wait_time = getattr(e, 'retry_after', backoff_factor ** retries)
        print(f"Rate limit hit. Sleeping for {wait_time} seconds before retrying...")
        time.sleep(wait_time)
        retries += 1

if retries == max_retries:
    print("Max retries reached. Some tweets may be missing.")

df = pd.DataFrame(tweetList, columns=["text"])

In [0]:
# import pandas as pd
# import time
# import tweepy

# tweetList = []
# max_retries = 5
# backoff_factor = 2

# # Assuming 'tweets' is a Tweepy Paginator or iterable that may raise TooManyRequests

# retries = 0
# while retries < max_retries:
#     try:
#         for tweet in tweets:
#             tweetList.append(tweet.text)
#         break  # Exit retry loop if successful
#     except tweepy.TooManyRequests as e:
#         wait_time = getattr(e, 'retry_after', backoff_factor ** retries)
#         print(f"Rate limit hit. Sleeping for {wait_time} seconds before retrying...")
#         time.sleep(wait_time)
#         retries += 1

# if retries == max_retries:
#     print("Max retries reached. Some tweets may be missing.")

# df = pd.DataFrame(tweetList, columns=["text"])
# # display(df)

In [0]:
import demoji
import re
 
def cleanTweetText(tweets):
  tweetsText = []
  for i in tweets:
    # remove emojis
    tweet = demoji.replace(i)
    # remove URLs
    tweetNoURL = re.sub(r'http\S+', '', tweet)
    tweetsText.append(tweetNoURL)
  return tweetsText
 
df["text"] = cleanTweetText(df["text"])

In [0]:
import spacy
import string
from spacy.tokenizer import Tokenizer
import string
from string import digits
 
nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)
punct = str.maketrans('', '', string.punctuation)
remove_digits = str.maketrans('', '', digits)
 
def getTokens(text):
  tokens = []
  for doc in tokenizer.pipe(text):
    doc_tokens = []    
    for token in doc:
      if token.text.lower().startswith('@') or token.text.lower().startswith('\\') or '_' in token.text.lower() or 'rt' in token.text.lower() or '‰' in token.text.lower():
        continue
      token_no_punc = str(token.text).translate(punct).translate(remove_digits)
      if len(token_no_punc)>1:
        doc_tokens.append(token_no_punc)
    tokens.append(doc_tokens)
  return tokens
 
df["tokens_as_str"] = [' '.join(map(str, i)) for i in getTokens(df["text"])]

In [0]:
import numpy as np

df = df.replace(
    r'^\s*$',
    np.nan,
    regex=True
)
df = df.dropna()
df = df.drop_duplicates(
    subset=['text']
)

if not df.empty:
    display(df)
else:
    print("DataFrame is empty after cleaning.")

In [0]:
if not df.empty:
    updates = spark.createDataFrame(df)
    updates.createOrReplaceGlobalTempView("updates")
else:
    print("DataFrame is empty. No Spark DataFrame created.")

In [0]:
write_format = 'delta'
save_path = 'dbfs:/mnt/FileStore/bronze/tweets.delta'
table_name = 'bronze.tweeter_bronze.tweeter'
 
# Create the table if it does not already exist.
spark.sql("CREATE TABLE IF NOT EXISTS bronze.tweeter_bronze.tweeter USING DELTA")